# ws 04 BERT

In [ ]:
!pip install ktrain -q

In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
ktrain.__version__   # 0.28.2

In [ ]:
!nvidia-smi

In [ ]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')

%ls '/content/drive/MyDrive/Colab Notebooks/datasets'

In [ ]:
uri = '/content/drive/MyDrive/Colab Notebooks/datasets/IMDB Dataset.csv'
df = pd.read_csv(uri, encoding='utf-8')
df.head()

In [ ]:
df.sentiment.unique()

In [ ]:
df.groupby('sentiment').count()

In [ ]:
_, class_names = pd.factorize(df.sentiment)
class_names.to_list()

In [ ]:
idx.shape

Train test 

In [ ]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df.review, df.sentiment, 
#                                                     test_size=.5, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(df.review[:2000],
                                                    df.sentiment[:2000], 
                                                    test_size=.5, random_state=1)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
# Transformer Model
pretrain_model = 'distilbert-base-uncased'
transformer = text.Transformer(pretrain_model, maxlen=250,  # 400
                               classes=class_names.to_list()) #  = [0,1])

In [ ]:
train = transformer.preprocess_train(X_train.to_list(), y_train.to_list())
test = transformer.preprocess_test(X_test.to_list(), y_test.to_list())

In [ ]:
train.x.shape

In [ ]:
train.x[:2]

In [ ]:
train.y.shape

Model

In [ ]:
model = transformer.get_classifier()

In [ ]:
model.summary()

In [ ]:
learner = ktrain.get_learner(model, train_data=train, 
                             val_data=test,
                             batch_size=16)

In [ ]:
learner.lr_find()  

In [ ]:
learner.lr_plot()

In [ ]:
learner.fit_onecycle(2e-5, 1) #

In [ ]:
# transformer.get_classes

In [ ]:
learner.validate(class_names=transformer.get_classes()) 

In [ ]:
learner.view_top_losses(n=3, preproc=transformer) # 

In [ ]:
test.x[3629]

In [ ]:
X_test[3629]

In [ ]:
X_test[22232]

In [ ]:
test_preprocess = transformer.preprocess_test(['this movie is great'])[0]
test_preprocess

Prediction

In [ ]:
predictor = ktrain.get_predictor(learner.model, transformer) 

In [ ]:
new_reviews = [ 'This movie was horrible! The plot was boring. Acting was okay, though.',
         'The film really sucked. I want my money back.',
        'The plot had too many holes.',
        'What a beautiful romantic comedy. 10/10 would see again!', ]

comment1 = 'this film is about politics. waste my time and money.'
comment2 = 'this film is about politics. It sucks, horrible. Can i get money back'
comment3 = 'this film is about politics. i give 5 stars'
comment4 = 'this film is about politics. you should watch it.'

new_reviews = [comment1, comment2, comment3, comment4]   

In [ ]:
predictor.predict(new_reviews)

In [ ]:
preds = predictor.predict(new_reviews, return_proba=True)
print(preds)

In [ ]:
results = [np.argmax(p) for p in preds]
results

In [ ]:
predictor.predict(X_test[3629])

 save the model

In [ ]:
predictor.save('my_predictor')
reload_predictor = ktrain.load_predictor('my_predictor')

In [ ]:
# copy to Gdrive
# Mount drive
# !cp -r 'my_predictor' '/content/drive/MyDrive/Colab Notebooks/distilBERT_imdb'

In [ ]:
# !ls '/content/drive/MyDrive/Colab Notebooks/distilBERT_imdb'